In [ ]:
import os
import warnings

warnings.filterwarnings(action='ignore', message='numpy.dtype size changed')
warnings.filterwarnings(action='ignore', message='compiletime version 3.5 of module')

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
os.chdir(os.path.split(workbookDir)[0])

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from synthesized.testing.evaluation import Evaluation
from synthesized.testing.synthetic_distributions import *

#evaluation = Evaluation(config_path='../synthesized/configs/evaluation/synthetic_distributions.json', name='synthetic')
evaluation = Evaluation()

### Gauss "ball" outside of center

In [ ]:
data = create_gauss_ball(x_mean=1000, x_std=100, y_mean=100, y_std=10, size=10000)

In [ ]:
synthesize_and_plot(data, name='ball', evaluation=evaluation)

### Gauss "ball" around of zero

In [ ]:
data = create_gauss_ball(x_mean=0, x_std=100, y_mean=0, y_std=10, size=10000)

In [ ]:
synthesize_and_plot(data, name='ball_ext', evaluation=evaluation)

### Line of noise that far from zero

In [ ]:
data = create_gauss_line(x_range=(0, 1000), intercept=100, slope=-0.1, y_std=10, size=10000)

In [ ]:
synthesize_and_plot(data, name='line', evaluation=evaluation)

### Line of noise that comes from zero

In [ ]:
data = create_gauss_line(x_range=(0, 1000), intercept=0, slope=0.1, y_std=10, size=10000)

In [ ]:
synthesize_and_plot(data, name='line_ext', evaluation=evaluation)

### Power law distribution

In [ ]:
data = create_power_law_distribution(shape=0.5, scale=1000, size=10000)

In [ ]:
synthesize_and_plot(data, name='power_law', evaluation=evaluation)

### Conditional distribution

In [ ]:
data = create_conditional_distibution((10,2), (20,5), (30,1), size=10000)

In [ ]:
synthesize_and_plot(data, name='conditional', evaluation=evaluation)

### Bernoulli distribution

In [ ]:
data = create_bernoulli_distribution(ratio=0.5, size=10000)

In [ ]:
synthesize_and_plot(data, name='bernoulli_50/50', evaluation=evaluation)

In [ ]:
data = create_bernoulli_distribution(ratio=0.2, size=10000)

In [ ]:
synthesize_and_plot(data, name='bernoulli_20/80', evaluation=evaluation)

### Categorical distribution

In [ ]:
data = create_unifom_categorical(n_classes=10, size=10000)

In [ ]:
synthesize_and_plot(data, name='categorical_uniform', evaluation=evaluation)

In [ ]:
data = create_power_law_categorical(n_classes=10, size=10000)

In [ ]:
synthesize_and_plot(data, name='categorical_powerlaw', evaluation=evaluation)

In [ ]:
evaluation.write_metrics()